<a href="https://colab.research.google.com/github/Francisss3/AAI612_Francis/blob/main/Copy_of_Notebook4_4_mini_project1_fruits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# AAI612: Deep Learning & its Applications

*Notebook 4.3: Graded Assignment: Mini Project I*

<a href="https://colab.research.google.com/github/harmanani/AAI612/blob/main/Week4/Notebook4.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assessment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/dataset_fruits.zip -d /content/dataset_fruits

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset_fruits/dataset/dataset/train/rottenoranges/saltandpepper_Screen Shot 2018-06-12 at 11.22.47 PM.png  
  inflating: /content/dataset_fruits/__MACOSX/dataset/dataset/train/rottenoranges/._saltandpepper_Screen Shot 2018-06-12 at 11.22.47 PM.png  
  inflating: /content/dataset_fruits/dataset/dataset/train/rottenoranges/translation_Screen Shot 2018-06-12 at 11.38.19 PM.png  
  inflating: /content/dataset_fruits/__MACOSX/dataset/dataset/train/rottenoranges/._translation_Screen Shot 2018-06-12 at 11.38.19 PM.png  
  inflating: /content/dataset_fruits/dataset/dataset/train/rottenoranges/rotated_by_75_Screen Shot 2018-06-12 at 11.37.00 PM.png  
  inflating: /content/dataset_fruits/__MACOSX/dataset/dataset/train/rottenoranges/._rotated_by_75_Screen Shot 2018-06-12 at 11.37.00 PM.png  
  inflating: /content/dataset_fruits/dataset/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.35.51 PM.png  
  i

In [ ]:
!ls /content/dataset_fruits/dataset


dataset  test  train


In [ ]:
!ls /content/dataset_fruits/dataset/train
!ls /content/dataset_fruits/dataset/test
!ls /content/dataset_fruits/dataset/dataset


freshapples  freshbanana  freshoranges	rottenapples  rottenbanana  rottenoranges
freshapples  freshbanana  freshoranges	rottenapples  rottenbanana  rottenoranges
test  train


In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning.

## The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. Download the dataset from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification). The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

![image.png](attachment:4c8c02c9-0cbe-4048-8d01-cdd5e3cf3fe6.png)<img src="./images/fruits.png" style="width: 600px;">

## Load ImageNet Base Model

Start with a model pretrained on `ImageNet`. Load the model with the correct weights, set an input shape, and choose to remove the last layers of the model. Remember that images have three dimensions: a height, and width, and a number of channels. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. This cannot be changed or the assessment will fail. If you need a reference for setting up the pretrained model, please take a look at [Notebook 4.2](https://github.com/harmanani/AAI612/blob/main/Week4/Notebook%204.2.ipynb) where we implemented transfer learning.

In [ ]:
import ssl
from tensorflow import keras

ssl._create_default_https_context = ssl._create_unverified_context

base_model = keras.applications.VGG16(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Freeze Base Model

Next, we suggest freezing the base model. This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [ ]:
# Freeze base model
base_model.trainable = False


## Add Layers to Model

Now it's time to add layers to the pretrained model. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.  You may add more layers than specified below.

In [ ]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)

# Add pooling layer or flatten layer (using GlobalAveragePooling2D here)
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer with 6 neurons for the 6 fruit categories
outputs = keras.layers.Dense(6, activation='softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs, outputs)


In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,717,766 (56.14 MB)

 Trainable params: 3,078 (12.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


## Augment the Data

If you'd like, try to augment the data to improve the dataset. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created).

In [ ]:
# Training data generator (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data generator (NO augmentation, only rescaling)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Load training dataset
train_it = train_datagen.flow_from_directory(
    '/content/dataset_fruits/dataset/train',
    target_size=(224, 224),
    batch_size=32,  # Ensure batch size is defined
    color_mode='rgb',
    class_mode="categorical"
)

# Load validation dataset
valid_it = valid_datagen.flow_from_directory(
    '/content/dataset_fruits/dataset/test',
    target_size=(224, 224),
    batch_size=32,
    color_mode='rgb',
    class_mode="categorical"
)


Found 10901 images belonging to 6 classes.
Found 2698 images belonging to 6 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting your desired number of epochs.

In [ ]:
import math

# Calculate steps per epoch
steps_per_epoch = math.ceil(train_it.samples / train_it.batch_size)
validation_steps = math.ceil(valid_it.samples / valid_it.batch_size)

# Train the model
history = model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=10
)


Epoch 1/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 152s 447ms/step - accuracy: 0.5424 - loss: 1.4654 - val_accuracy: 0.7728 - val_loss: 1.1496
Epoch 2/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 442ms/step - accuracy: 0.7016 - loss: 1.1676 - val_accuracy: 0.7665 - val_loss: 0.9686
Epoch 3/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 150s 441ms/step - accuracy: 0.7698 - loss: 1.0166 - val_accuracy: 0.8084 - val_loss: 0.8389
Epoch 4/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 150s 441ms/step - accuracy: 0.7985 - loss: 0.8982 - val_accuracy: 0.8158 - val_loss: 0.7515
Epoch 5/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 150s 441ms/step - accuracy: 0.8089 - loss: 0.8119 - val_accuracy: 0.8340 - val_loss: 0.6801
Epoch 6/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 442ms/step - accuracy: 0.8192 - loss: 0.7456 - val_accuracy: 0.8106 - val_loss: 0.6352
Epoch 7/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 443ms/step - accuracy: 0.8309 - loss: 0.6849 - val_accuracy: 0.8462 - val_loss: 0.5751
Epoch 8/10
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 442ms/step - accuracy: 0.8357 -

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [ ]:
# Unfreeze the base model
base_model.trainable = True  # Unfreezing VGG16 for fine-tuning

# Compile the model with a very low learning rate
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),  # Very low LR for fine-tuning
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Fine-tune the model
history_finetune = model.fit(
    train_it,
    validation_data=valid_it,
    steps_per_epoch=math.ceil(train_it.samples / train_it.batch_size),
    validation_steps=math.ceil(valid_it.samples / valid_it.batch_size),
    epochs=5  # Fine-tuning for a few more epochs
)

Epoch 1/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 175s 474ms/step - accuracy: 0.9028 - loss: 0.2786 - val_accuracy: 0.9900 - val_loss: 0.0314
Epoch 2/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 153s 449ms/step - accuracy: 0.9835 - loss: 0.0509 - val_accuracy: 0.9956 - val_loss: 0.0137
Epoch 3/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 153s 448ms/step - accuracy: 0.9908 - loss: 0.0321 - val_accuracy: 0.9970 - val_loss: 0.0084
Epoch 4/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 443ms/step - accuracy: 0.9943 - loss: 0.0153 - val_accuracy: 0.9985 - val_loss: 0.0047
Epoch 5/5
341/341 ━━━━━━━━━━━━━━━━━━━━ 151s 442ms/step - accuracy: 0.9971 - loss: 0.0105 - val_accuracy: 0.9981 - val_loss: 0.0045


## Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation.

## Discussion

In [ ]:
from IPython.display import display, Markdown

discussion_text = """
### Discussion

Initially, the model was trained using a pre-trained VGG16 base with frozen layers and data augmentation applied to the training set. While this approach helped improve generalization, the validation accuracy plateaued at around 87.99%, falling short of the required 92%. This indicated that the model had learned useful features but required further refinement to reach the desired performance.

To improve accuracy, fine-tuning was applied by unfreezing the base model and training with a significantly lower learning rate. This allowed the model to adjust the pre-trained VGG16 features specifically for the fresh and rotten fruit classification task while avoiding drastic weight updates that could erase previously learned representations. As a result, the validation accuracy steadily improved, reaching 99.81% after just five fine-tuning epochs, with a corresponding decrease in validation loss.

This demonstrates the effectiveness of transfer learning combined with fine-tuning, as the model was able to leverage prior knowledge from ImageNet while adapting to the new dataset. Additionally, data augmentation played a crucial role in preventing overfitting by exposing the model to varied versions of the training data.
"""

display(Markdown(discussion_text))



### Discussion

Initially, the model was trained using a pre-trained VGG16 base with frozen layers and data augmentation applied to the training set. While this approach helped improve generalization, the validation accuracy plateaued at around 87.99%, falling short of the required 92%. This indicated that the model had learned useful features but required further refinement to reach the desired performance.

To improve accuracy, fine-tuning was applied by unfreezing the base model and training with a significantly lower learning rate. This allowed the model to adjust the pre-trained VGG16 features specifically for the fresh and rotten fruit classification task while avoiding drastic weight updates that could erase previously learned representations. As a result, the validation accuracy steadily improved, reaching 99.81% after just five fine-tuning epochs, with a corresponding decrease in validation loss. 

This demonstrates the effectiveness of transfer learning combined with fine-tuning, as the model was able to leverage prior knowledge from ImageNet while adapting to the new dataset. Additionally, data augmentation played a crucial role in preventing overfitting by exposing the model to varied versions of the training data.
